In [102]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator
from collections import Counter

In [ ]:
#!python -m spacy download en_core_web_sm
#!python -m nltk.downloader wordnet
#!python -m nltk.downloader omw

In [4]:
# This needs to be defined by the drop down in the UI
subgroup1 = "woman"
subgroup2 = "man"
subgroup3 = "non-binary"

In [5]:
data = load_dataset("c4", "en", split= "train", streaming = True)

In [6]:
grab_n = 10000
# For streaming data
print('Note: Just taking the first %s instances.' % grab_n)
data_head = data.take(grab_n)
#data_head = [["there is a woman with a hairbrush"],["there is a woman with a hairbrush"],["there is a woman with a hairbrush"],["there is a man with a dog"],["there is a man with a dog"]]
df = pd.DataFrame(data_head, columns=["text"])
# If not streaming, use:
#df = pd.json_normalize(data)

Note: Just taking the first 10000 instances.


In [7]:
def count_vocab_frequencies(df):
    """
    Based on an input pandas DataFrame with a 'text' column, 
    this function will count the occurrences of all words
    with a frequency higher than 'cutoff' and will return another DataFrame
    with the rows corresponding to the different vocabulary words
    and the column to the count count of that word.
    """
    # Move this up as a constant in larger code.
    batch_size = 10
    
    # We do this to calculate per-word statistics
    df['text'] = df['text'].str.lower()
    # Regex for pulling out single words
    cvec = CountVectorizer(token_pattern=u"(?u)\\b\\w+\\b", lowercase=True)
    
    # We also do this because we need to have the tokenization per sentence 
    # so that we can look at co-occurrences of words across sentences for nPMI calculation
    sent_tokenizer = cvec.build_tokenizer()
    df['tokenized'] = df.text.apply(sent_tokenizer)
    
    # Fast calculation of single word counts
    cvec.fit(df.text)
    document_matrix = cvec.transform(df.text)
    batches = np.linspace(0, df.shape[0], batch_size).astype(int)
    i = 0
    tf = []
    while i < len(batches) - 1:
        batch_result = np.sum(document_matrix[batches[i]:batches[i+1]].toarray(), axis=0)
        tf.append(batch_result)
        i += 1
    term_freq_df = pd.DataFrame([np.sum(tf, axis=0)], columns=cvec.get_feature_names()).transpose()
    
    # Now organize everything into the dataframes
    term_freq_df.columns = ['count']
    term_freq_df.index.name = 'word'
    sorted_term_freq_df = pd.DataFrame(term_freq_df.sort_values(by='count', ascending=False)['count'])
    return sorted_term_freq_df, df

In [8]:
term_df, df = count_vocab_frequencies(df)
# p(word).  Note that multiple occurrences of a word in a sentence increases its probability.
# We may want to do something about that.
term_df['proportion'] = term_df['count']/float(sum(term_df['count']))
# Sanity check
print(term_df.head())
print(term_df.tail())

       count  proportion
word                    
the   186019    0.050628
and   107893    0.029365
to    103090    0.028058
of     89417    0.024336
a      81307    0.022129
             count    proportion
word                            
interestel       1  2.721674e-07
interethnic      1  2.721674e-07
interfaced       1  2.721674e-07
interfacing      1  2.721674e-07
𐌼𐌿𐌽𐌳𐍃            1  2.721674e-07


In [9]:
def get_PMI(df_coo, subgroup):
    # PMI(x;y) = h(y) - h(y|x)
    #          = h(subgroup) - h(subgroup|word)
    #          = log (p(subgroup|word) / p(subgroup))
    # nPMI additionally divides by -log(p(x,y)) = -log(p(x|y)p(y))

    # Calculation of p(subgroup)
    subgroup_prob = term_df.loc[subgroup]['proportion']
    # Apply a function to all words to calculate log p(subgroup|word)
    # The word is indexed by mlb.classes_ ; 
    # we pull out the word using the mlb.classes_ index and then get its count using our main term_df
    pmi_df = pd.DataFrame(df_coo.apply(lambda x: np.log(x.values/term_df.loc[mlb.classes_[x.index]]['count']/subgroup_prob)))
    # If all went well, this will be correlated with high frequency words
    # Until normalizing
    return pmi_df

In [10]:
def get_nPMI(pmi_df, df_coo):
    normalize_df = pd.DataFrame(df_coo.apply(lambda x: -np.log(x.values/term_df.loc[mlb.classes_[x.index]]['count'] * term_df.loc[mlb.classes_[x.index]]['proportion'])))
    npmi_df = pmi_df/normalize_df
    return npmi_df

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

# Makes a sparse vector (shape: # sentences x # words),
# with the count of each word per sentence.
#    the a cat poop woman Blind
# 0 
# 1
mlb = MultiLabelBinarizer()
df_mlb = pd.DataFrame(mlb.fit_transform(df['tokenized']))
npmi_df_pair = pd.DataFrame(columns=[subgroup1, subgroup2])
pmi_df_pair = pd.DataFrame(columns=[subgroup1, subgroup2])
for subgroup in (subgroup1, subgroup2):
    # Index of the subgroup word in the sparse vector
    subgroup_idx = np.where(mlb.classes_ == subgroup)[0][0]
    # Dataframe for the subgroup (with counts)
    df_subgroup = df_mlb.iloc[:, subgroup_idx]
    # Create cooccurence matrix for the given subgroup and all other words.
    # Note it also includes the word itself, so that count should be subtracted 
    # (the word will always co-occur with itself)
    df_coo = pd.DataFrame(df_mlb.T.dot(df_subgroup))#.drop(index=subgroup_idx, axis=1)
    pmi_df = get_PMI(df_coo, subgroup)
    pmi_df_pair[subgroup] = pmi_df
    npmi_df = get_nPMI(pmi_df, df_coo)
    npmi_df_pair[subgroup] = npmi_df

/home/sasha/miniconda3/envs/datametrics/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
# woman - man: If it's negative, it's man-biased; if it's positive, it's woman positive.
npmi_bias = pd.DataFrame(npmi_df_pair[subgroup1] - npmi_df_pair[subgroup2])
pmi_bias = pd.DataFrame(pmi_df_pair[subgroup1] - pmi_df_pair[subgroup2])

In [13]:
# Words that only occur with one or the other -- needed for PMI
s1_only_words = pmi_bias[pmi_bias[0].values==np.inf]
s2_only_words = pmi_bias[pmi_bias[0].values==-np.inf]

# Filter
npmi_bias_filtered = npmi_bias.dropna() # [(np.inf > npmi_bias[0]) & (npmi_bias[0] > -np.inf)].sort_values(by=[0])

In [14]:
n = 50

In [15]:
print("@%s, the %s bias is:\t%.2f" % (n, subgroup2, np.abs(sum(npmi_bias_filtered[:n].values))))
print("@%s, the %s bias is:\t%.2f" % (n, subgroup1, sum(npmi_bias_filtered[-n:].values)))

@50, the man bias is:	0.28
@50, the woman bias is:	2.54


In [ ]:
print("Top %s most %s-biased words" % (n, subgroup2))
npmi_bias_filtered.sort_values(by=[0], ascending=True)[:n]

In [ ]:
print("Top %s most %s-biased words" % (n,subgroup1))
npmi_bias_filtered.sort_values(by=[0], ascending=True)[-n:].sort_values(by=[0], ascending=False)

## Trying out Spacy + WordNet

In [75]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacy_wordnet", after='tagger', config={'lang': nlp.lang})
#nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

In [62]:
womanwords=npmi_bias_filtered.sort_values(by=[0], ascending=True)[-n:].sort_values(by=[0], ascending=False).index.tolist()

In [79]:
#Docs: https://github.com/recognai/spacy-wordnet/

In [111]:
for token in womanwords[:5]:
    print(token)
    # We get those synsets within the desired domains
    token= nlp(token)[0]
    synsets = token._.wordnet.synsets()
    for s in synsets:
        print()

woman
woman.n.01
woman.n.02
charwoman.n.01
womanhood.n.02
quit
discontinue.v.01
leave_office.v.01
depart.v.04
foreswear.v.02
drop_out.v.01
swedish
swedish.n.01
swedish.a.01
complement
complement.n.01
complement.n.02
complement.n.03
complement.n.04
complement.n.05
complement.n.06
complement.v.01
vitamins
vitamin.n.01


In [126]:
syns=[]
for token in womanwords:
    # We get those synsets within the desired domains
    token= nlp(token)[0]
    synsets = token._.wordnet.synsets()
    for s in synsets:
        syns.append(s.name().split('.')[0])
        
print(Counter(syns))

Counter({'shoot': 14, 'representation': 9, 'flight': 9, 'down': 8, 'complement': 7, 'chatter': 6, 'chamber': 5, 'fear': 4, 'booster': 4, 'scarf': 4, 'overlap': 4, 'coaster': 3, 'murray': 3, 'helpless': 3, 'birth': 3, 'ritual': 3, 'woman': 2, 'swedish': 2, 'inject': 2, 'dress': 2, 'strengthening': 2, 'strengthen': 2, 'scourge': 2, 'testify': 2, 'dumb': 2, 'cornell': 2, 'epidemic': 2, 'charwoman': 1, 'womanhood': 1, 'discontinue': 1, 'leave_office': 1, 'depart': 1, 'foreswear': 1, 'drop_out': 1, 'vitamin': 1, 'theatrical_performance': 1, 'miraculously': 1, 'activism': 1, 'blast': 1, 'film': 1, 'dart': 1, 'tear': 1, 'photograph': 1, 'fritter': 1, 'childbirth': 1, 'training': 1, 'dressing': 1, 'prepare': 1, 'groom': 1, 'tone': 1, 'malaysia': 1, 'workplace': 1, 'escape': 1, 'trajectory': 1, 'fledge': 1, 'urgently': 1, 'desperately': 1, 'bane': 1, 'terror': 1, 'flagellate': 1, 'lay_waste_to': 1, 'reverence': 1, 'alternately': 1, 'preconceive': 1, 'preconceived': 1, 'parturition': 1, 'parenta